In [1]:
import pandas as pd
import plotly.express as px 
import plotly.graph_objects as go

import dash  
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [2]:
# Import and clean data (importing csv into pandas)
df = pd.read_csv("MM_completed_phase2n3.csv")

df.columns = df.columns.str.lower().str.replace(' ', '_')
df.head()

,rank,nct_number,title,acronym,status,study_results,conditions,interventions,outcome_measures,sponsor/collaborators,...,other_ids,start_date,primary_completion_date,completion_date,first_posted,results_first_posted,last_update_posted,locations,study_documents,url
0,1,NCT00657488,Thalidomide 100 mg/Day Versus Thalidomide 400 ...,NaN,Completed,No Results Available,Multiple Myeloma,Drug: Thalidomide,all cause mortality|response rate,Celgene,...,LATH0102/IFM0102,"December 1, 2001","October 1, 2005","April 1, 2006","April 14, 2008",NaN,"October 18, 2019","Hopital St Joseph, Gilly, Belgium|UCL de Mont ...",NaN,https://ClinicalTrials.gov/show/NCT00657488
1,2,NCT02419118,"""Monoclonal Antibodies for Treatment of Multip...",DARA,Completed,No Results Available,Multiple Myeloma,Drug: Daratumumab|Drug: Lenalidomide|Drug: Dex...,Response by IMWG criteria|T cell subset numbers,Vejle Hospital,...,181114,January 2015,"October 1, 2015","October 1, 2015","April 17, 2015",NaN,"November 7, 2017","Department of Hematology Vejle Hospital, Vejle...",NaN,https://ClinicalTrials.gov/show/NCT02419118
2,3,NCT00038090,Thalidomide-Dexamethasone for Multiple Myeloma,NaN,Completed,No Results Available,Multiple Myeloma,Drug: Thalidomide|Drug: Dexamethasone,Response Rate,M.D. Anderson Cancer Center|Celgene Corporation,...,ID00-070,June 2000,May 2005,May 2005,"May 29, 2002",NaN,"October 31, 2018",University of Texas M. D. Anderson Cancer Cent...,NaN,https://ClinicalTrials.gov/show/NCT00038090
3,4,NCT00514371,A Study of Tanespimycin (KOS-953) in Patients ...,TIME-2,Completed,No Results Available,Multiple Myeloma,Drug: tanespimycin|Drug: Bortezomib,Objective Response Rate|Response rate between ...,Bristol-Myers Squibb,...,CA200-003|KAG-302,August 2007,February 2009,February 2009,"August 9, 2007",NaN,"December 7, 2015","Local Institution, Berkeley, California, Unite...",NaN,https://ClinicalTrials.gov/show/NCT00514371
4,5,NCT00090493,Study of MAGE-A3 and NY-ESO-1 Immunotherapy in...,NaN,Completed,Has Results,Multiple Myeloma,Biological: MAGE-A3|Biological: MAGE-A3 AND NY...,The Number of Participants Experiencing a Resp...,University of Arkansas,...,UARK 2003-26,June 2004,July 2007,May 2012,"August 31, 2004","June 24, 2013","June 24, 2013",University of Arkansas for Medical Sciences/MI...,NaN,https://ClinicalTrials.gov/show/NCT00090493


In [3]:
df['completion_year'] = pd.DatetimeIndex(df['completion_date']).year
df.completion_year = df.completion_year.apply(lambda x: str(x).replace('.0',''))

In [4]:
df['industry_sponsor'] = df.funded_bys.str.contains('Industry').astype(int)
df.industry_sponsor.head()

0    1
1    0
2    1
3    1
4    0
Name: industry_sponsor, dtype: int64

In [5]:
app = dash.Dash(__name__)

In [6]:
# App layout
app.layout = html.Div([

    html.H1("Web Application Dashboards with Dash", style={'text-align': 'center'}),

    html.Div([
        html.Label('Study Phase'),
        dcc.Dropdown(id="slct_phase",
                 options=[
                     {"label": "Phase 2", "value": "Phase 2"},
                     {"label": "Phase 3", "value": "Phase 3"},
                     {"label": "Phase 1/2", "value": "Phase 1|Phase 2"},
                     {"label": "Phase 2/3", "value": "Phase 2|Phase 3"}],
                 multi=False,
                 #value="Phase 2",
                 placeholder="Select phase of the trials",
                 style={'width': "40%"}
                 ),
        
        html.Label('Sponsor Type'),
        dcc.Dropdown(id="slct_spons",
                 options=[
                     {"label": "All Sponsor Types", "value": "All"},
                     {"label": "Industry Sponsors", "value": "Industry"}],
                 multi=False,
                 #value="All",
                 placeholder="Select type of trial sponsor",
                 style={'width': "40%"}
                 ),
        
        html.Label('Age Group'),
        dcc.RangeSlider(
            min=0,
            max=100,
            step=1,
            marks={
                0: '0 ',
                18: '18',
                40: '40',
                65: '65',
                100: '100'
            },
            value=[18, 100],
        ),
        #html.Div(id='output_container_slide', children=[])
             ]),  # , style={'columnCount': 3}

    html.Div(id='output_container', children=[]),
    html.Br(),

    dcc.Graph(id='trial_en_graph', figure={})

])


# ------------------------------------------------------------------------------
# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='trial_en_graph', component_property='figure')],
     #Output(component_id='output_container_slide', component_property='children')],
    [Input(component_id='slct_phase', component_property='value'),
     Input(component_id='slct_spons', component_property='value')]
     #Input(component_id='range_slider', component_property='value')]
)

# Business logic
def update_graph(option_slctd, spons_slctd):
    print(option_slctd)
    print(spons_slctd)

    container = "The phase chosen by user was: {}".format(option_slctd)

    dff = df.copy()
    dff = dff[dff["phases"] == option_slctd]
    #dff = dff[dff["phases"].isin(list(option_slctd))]
    if spons_slctd=='Industry':
        dff = dff[dff["industry_sponsor"] == 1]
    dff = dff[['completion_year','enrollment']].groupby('completion_year',as_index=False).sum()
    print(dff.head())
    
    fig = px.bar(dff, y='enrollment', x='completion_year')
    
    #container_slider = 'You have selected "{}"'.format(range_slider)
    
    return container, fig


# ------------------------------------------------------------------------------
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Sep/2020 20:31:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2020 20:31:46] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2020 20:31:46] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2020 20:31:46] "GET /_favicon.ico?v=1.16.0 HTTP/1.1" 200 -


None
None
Empty DataFrame
Columns: [completion_year, enrollment]
Index: []


127.0.0.1 - - [07/Sep/2020 20:31:46] "POST /_dash-update-component HTTP/1.1" 200 -


Phase 2
None
  completion_year  enrollment
0            1999         0.0
1            2000       133.0
2            2001       141.0
3            2002       224.0
4            2003        97.0


127.0.0.1 - - [07/Sep/2020 20:32:00] "POST /_dash-update-component HTTP/1.1" 200 -


Phase 2
Industry
  completion_year  enrollment
0            2001        10.0
1            2003         6.0
2            2004        60.0
3            2005       162.0
4            2006       302.0


127.0.0.1 - - [07/Sep/2020 20:32:02] "POST /_dash-update-component HTTP/1.1" 200 -
